# MCCoder MCEval Dataset for evaluation


## Preparation


### Generate JSON dataset file from excel, python source code and log file.


#### Generate Json file from python source file


In [7]:
import pandas as pd
import os
import json

# Load the Excel file
excel_file_path = 'docs/WMX3API_MCEval_Dataset.xlsx'
df = pd.read_excel(excel_file_path)

# Initialize an empty list to store the data for JSON
data_list = []

# Iterate through each row in the dataframe
for index, row in df.iterrows():
    task_id = row['TaskId']
    # Construct the path to the Python file
    py_file_path = f'MCEval_Files/{task_id}_MCEval.py'
    
    # Read the Python file to get the CanonicalCode
    if os.path.exists(py_file_path):
        with open(py_file_path, 'r') as file:
            canonical_code = file.read()
    else:
        canonical_code = ''
    
    # Create a dictionary for the current row
    row_data = row.to_dict()
    row_data['CanonicalCode'] = canonical_code
    
    # Append the dictionary to the list
    data_list.append(row_data)

# Construct the output JSON file path
json_file_path = excel_file_path.replace('.xlsx', '.json')

# Write the data to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

print(f"JSON file has been generated at {json_file_path}")


JSON file has been generated at docs/WMX3API_MCEval_Dataset.json


#### Generate Json file from python source file with motion log


In [ ]:
import pandas as pd
import os
import json
import socket
from time import *
from tqdm import tqdm

# Load the Excel file
excel_file_path = 'docs/WMX3API_MCEval_Dataset.xlsx'
df = pd.read_excel(excel_file_path)

# Initialize an empty list to store the data for JSON
data_list = []

# Function to modify the Python code with logging
def add_log_code(py_code, task_id, axis):
    lines = py_code.split('\n')
    modified_code = []
    i = 0
    # Add logging start code
    while i < len(lines):
        modified_code.append(lines[i])
        if '.StartHome' in lines[i]:
            i += 1
            while i < len(lines) and lines[i].strip() != '':
                modified_code.append(lines[i])
                i += 1
            if i < len(lines):
                modified_code.append(f'''
                                     
    # <log ---------------------------------------------------------------------------                                                                 
    WMX3Log = Log(Wmx3Lib)

    axislist = {axis}                           
    num = len(axislist)

    # Set Axis numbers and control variables of log
    cmLogIn_0 = CoreMotionLogInput()
    cmLogIn_0.axisSelection.axisCount = num
    for i in range(0, num):
        cmLogIn_0.axisSelection.SetAxis(i, axislist[i])

    # Control variables to log
    cmLogIn_0.axisOptions.commandPos = 1
    cmLogIn_0.axisOptions.feedbackPos = 0
    cmLogIn_0.axisOptions.commandVelocity = 0
    cmLogIn_0.axisOptions.feedbackVelocity = 0

    # Set up log time
    option = LogChannelOptions()
    option.samplingPeriodInCycles = 1
    option.samplingTimeMilliseconds = 1000000  

    ret=WMX3Log.SetLogOption(0, option)
    if ret!=0:
        print('SetLogOption error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.1)
    ret = WMX3Log.SetCustomLog(0,cmLogIn_0)
    if ret!=0:
        print('SetCustomLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.1)

    # Set log file address
    path_0 = LogFilePath()
    WMX3Log.GetLogFilePath(0)
    path_0.dirPath = "C:\\\\"
    path_0.fileName = f"{task_id}_MCEval_Log.txt"
    ret = WMX3Log.SetLogFilePath(0, path_0)
    if ret!=0:
        print('SetLogFilePath error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return

    # Stop log just in case logging is on.
    ret = WMX3Log.StopLog(0)
    sleep(0.01)

    # Start log
    ret = WMX3Log.StartLog(0)
    if ret!=0:
        print('StartLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.01)
    # log> ---------------------------------------------------------------------------   
    
                ''')
        i += 1

    # Add logging stop code
    i = len(lines) - 1
    while i >= 0:
        if '.SetServoOn' in lines[i]:
            while i >= 0 and lines[i].strip() != '':
                i -= 1
            if i >= 0:
                modified_code.insert(i + 1, '''
                                     
    # <log --------------------------------------------------------------------------- 
    sleep(0.1)                                                                    
    # Stop log
    ret = WMX3Log.StopLog(0)
    if ret!=0:
        print('StopLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return

    for axisNo in axislist:                                
        ret = Wmx3Lib_cm.home.StartHome(axisNo)
        if ret!=0:
            print('StartHome error code is ' + str(ret) + ': ' + Wmx3Lib_cm.ErrorToString(ret))
            return
        Wmx3Lib_cm.motion.Wait(axisNo)                                                                  
    # log> ---------------------------------------------------------------------------     
                                     
                ''')
            break
        i -= 1

    return '\n'.join(modified_code)

# Function to send code and log data to socket server
def SendCodeLog(code_data_log):
    # 1. Create socket
    tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # 2. Connect server
    server_addr = ("10.211.55.5", 8888)
    tcp_socket.connect(server_addr)
    print("Socket connected")

    import time

    # 3. Send data
    tcp_socket.send(code_data_log.encode("utf-8"))

    time.sleep(0.1)

    send_data = 'exit'
    tcp_socket.send(send_data.encode("utf-8"))

    # 4. Receive data from server
    tcp_remsg = tcp_socket.recv(1024)
    print(tcp_remsg.decode("utf-8"))

    # 5. Close socket
    tcp_socket.close()
    print("Socket end")

# Iterate through each row in the dataframe with a progress bar
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    task_id = row['TaskId']
    # Construct the path to the Python file
    py_file_path = f'MCEval_Files/{task_id}_MCEval.py'
    
    # Read the Python file to get the CanonicalCode
    if os.path.exists(py_file_path):
        with open(py_file_path, 'r') as file:
            py_code = file.read()
    else:
        py_code = ''
    
    #If it is about motion log
    print('endp: ' + str(row['EndPoint']))
    if (str(row['EndPoint']) != 'nan'):
        # Extract axis from the first row of 'EndPoint'
        try:
            axis = list(range(len(eval(row['EndPoint'])[0])))
            axis = eval(row['EndPoint'])[0]
            print("axisl: " + str(axis))
            if not axis:
                continue
        except Exception as e:
            print(f"Error processing axis for task {task_id}: {e}")
            continue

        # Add log code to the Python file content
        modified_code = add_log_code(py_code, task_id, axis)
    else:
        modified_code = py_code
    
    # Save the modified code to a new file
    modified_file_path = f'MCEval_Files/{task_id}_MCEval_WithLog.py'
    with open(modified_file_path, 'w') as file:
        file.write(modified_code)
    
    # Send the modified code to the socket server for each task
    SendCodeLog(modified_code)
    sleep(0.3)

    # Create a dictionary for the current row
    row_data = row.to_dict()
    row_data['CanonicalCode'] = py_code
    
    # Append the dictionary to the list
    data_list.append(row_data)

    sleep(0.2)

# Construct the output JSON file path
json_file_path = excel_file_path.replace('.xlsx', '.json')

# Write the data to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

print(f"JSON file has been generated at {json_file_path}")


#### Generate Json file from python source file with motion log, adding error stastics


In [66]:
import pandas as pd
import os
import json
import socket
from time import sleep
from tqdm import tqdm

# Load the Excel file
excel_file_path = 'docs/WMX3API_MCEval_Dataset.xlsx'
df = pd.read_excel(excel_file_path)

# Initialize an empty list to store the data for JSON
data_list = []

# Lists to store TaskIds with different errors
error_task_ids = []
syntax_error_task_ids = []

# Function to modify the Python code with logging
def add_log_code(py_code, task_id, axis):
    lines = py_code.split('\n')
    modified_code = []
    i = 0
    # Add logging start code
    while i < len(lines):
        modified_code.append(lines[i])
        if '.StartHome' in lines[i]:
            i += 1
            while i < len(lines) and lines[i].strip() != '':
                modified_code.append(lines[i])
                i += 1
            if i < len(lines):
                modified_code.append(f'''
    # <log ---------------------------------------------------------------------------                                                                 
    WMX3Log = Log(Wmx3Lib)

    axislist = {axis}                           
    num = len(axislist)

    # Set Axis numbers and control variables of log
    cmLogIn_0 = CoreMotionLogInput()
    cmLogIn_0.axisSelection.axisCount = num
    for i in range(0, num):
        cmLogIn_0.axisSelection.SetAxis(i, axislist[i])

    # Control variables to log
    cmLogIn_0.axisOptions.commandPos = 1
    cmLogIn_0.axisOptions.feedbackPos = 0
    cmLogIn_0.axisOptions.commandVelocity = 0
    cmLogIn_0.axisOptions.feedbackVelocity = 0

    # Set up log time
    option = LogChannelOptions()
    option.samplingPeriodInCycles = 1
    option.samplingTimeMilliseconds = 1000000  

    ret=WMX3Log.SetLogOption(0, option)
    if ret!=0:
        print('SetLogOption error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.1)
    ret = WMX3Log.SetCustomLog(0,cmLogIn_0)
    if ret!=0:
        print('SetCustomLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.1)

    # Set log file address
    path_0 = LogFilePath()
    WMX3Log.GetLogFilePath(0)
    path_0.dirPath = "C:\\\\"
    path_0.fileName = f"{task_id}_MCEval_Log.txt"
    ret = WMX3Log.SetLogFilePath(0, path_0)
    if ret!=0:
        print('SetLogFilePath error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return

    # Stop log just in case logging is on.
    ret = WMX3Log.StopLog(0)
    sleep(0.01)

    # Start log
    ret = WMX3Log.StartLog(0)
    if ret!=0:
        print('StartLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return
    sleep(0.01)
    # log> ---------------------------------------------------------------------------   
    
                ''')
        i += 1

    # Add logging stop code
    i = len(lines) - 1
    while i >= 0:
        if '.SetServoOn' in lines[i]:
            while i >= 0 and lines[i].strip() != '':
                i -= 1
            if i >= 0:
                modified_code.insert(i + 1, '''
    # <log --------------------------------------------------------------------------- 
    sleep(0.1)                                                                    
    # Stop log
    ret = WMX3Log.StopLog(0)
    if ret!=0:
        print('StopLog error code is ' + str(ret) + ': ' + WMX3Log.ErrorToString(ret))
        return

    for axisNo in axislist:                                
        ret = Wmx3Lib_cm.home.StartHome(axisNo)
        if ret!=0:
            print('StartHome error code is ' + str(ret) + ': ' + Wmx3Lib_cm.ErrorToString(ret))
            return
        Wmx3Lib_cm.motion.Wait(axisNo)                                                                  
    # log> ---------------------------------------------------------------------------     
                                     
                ''')
            break
        i -= 1

    return '\n'.join(modified_code)

# Function to send code and log data to socket server
def SendCodeLog(code_data_log, task_id):
    # 1. Create socket
    tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # 2. Connect server
    server_addr = ("10.211.55.5", 8888)
    tcp_socket.connect(server_addr)
    print("Socket connected")

    # 3. Send data
    tcp_socket.send(code_data_log.encode("utf-8"))

    sleep(0.1)

    send_data = 'exit'
    tcp_socket.send(send_data.encode("utf-8"))

    # 4. Receive data from server
    tcp_remsg = tcp_socket.recv(1024).decode("utf-8")
    print(tcp_remsg)

    # Check for errors in the response
    if "error" in tcp_remsg:
        if "syntax error" in tcp_remsg:
            syntax_error_task_ids.append(task_id)
        else:
            error_task_ids.append(task_id)

    # 5. Close socket
    tcp_socket.close()
    print("Socket end")

# Iterate through each row in the dataframe with a progress bar
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    task_id = row['TaskId']
    # Construct the path to the Python file
    py_file_path = f'MCEval_Files/{task_id}_MCEval.py'
    
    # Read the Python file to get the CanonicalCode
    if os.path.exists(py_file_path):
        with open(py_file_path, 'r') as file:
            py_code = file.read()
    else:
        py_code = ''
    
    # If it is about motion log
    print('endp: ' + str(row['EndPoint']))
    if str(row['EndPoint']) != 'nan':
        # Extract axis from the first row of 'EndPoint'
        try:
            axis = list(range(len(eval(row['EndPoint'])[0])))
            axis = eval(row['EndPoint'])[0]
            print("axisl: " + str(axis))
            if not axis:
                continue
        except Exception as e:
            print(f"Error processing axis for task {task_id}: {e}")
            continue

        # Add log code to the Python file content
        modified_code = add_log_code(py_code, task_id, axis)
    else:
        modified_code = py_code
    
    # Save the modified code to a new file
    modified_file_path = f'MCEval_Files/{task_id}_MCEval_WithLog.py'
    with open(modified_file_path, 'w') as file:
        file.write(modified_code)
    
    # Send the modified code to the socket server for each task
    SendCodeLog(modified_code, task_id)
    sleep(0.3)

    # Create a dictionary for the current row
    row_data = row.to_dict()
    row_data['CanonicalCode'] = py_code
    
    # Append the dictionary to the list
    data_list.append(row_data)

    sleep(0.2)

# Construct the output JSON file path
json_file_path = excel_file_path.replace('.xlsx', '.json')

# Write the data to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

# Calculate accuracy
total_tasks = len(df)
error_count = len(error_task_ids)
syntax_error_count = len(syntax_error_task_ids)
success_count = total_tasks - (error_count + syntax_error_count)
accuracy = (success_count / total_tasks) * 100

print(f"JSON file has been generated at {json_file_path}")
print(f"Total tasks: {total_tasks}")
print(f"Error tasks: {error_count}")
print(f"Syntax error tasks: {syntax_error_count}")
print(f"Successful tasks: {success_count}")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Tasks with errors: {error_task_ids}")
print(f"Tasks with syntax errors: {syntax_error_task_ids}")


/opt/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Sparkline Group extension is not supported and will be removed
  for idx, row in parser.parse():
Processing rows:   0%|          | 0/69 [00:00<?, ?it/s]

endp: [[0],
[180]]
axisl: [0]
Socket connected
1
codedeta:Program begin.
Program End.
, Elapsed_time: 6.318
codeerr:
Socket end


Processing rows:   1%|▏         | 1/69 [00:06<07:51,  6.93s/it]

endp: [[1],
[200]]
axisl: [1]
Socket connected
2
codedeta:Program begin.
Program End.
, Elapsed_time: 3.985
codeerr:
Socket end


Processing rows:   3%|▎         | 2/69 [00:11<06:12,  5.56s/it]

...

endp: [[0,1,2],
[0,0,0]]
axisl: [0, 1, 2]
Socket connected
68
codedeta:Program begin.
Program End.
, Elapsed_time: 7.349
codeerr:
Socket end


Processing rows:  99%|█████████▊| 68/69 [07:51<00:06,  6.80s/it]

endp: [[0,1,2,3],
[0,0,0,100]]
axisl: [0, 1, 2, 3]
Socket connected
69
codedeta:Program begin.
Program End.
, Elapsed_time: 8.684
codeerr:
Socket end


Processing rows: 100%|██████████| 69/69 [07:58<00:00,  6.94s/it]

JSON file has been generated at docs/WMX3API_MCEval_Dataset.json
Total tasks: 69
Error tasks: 0
Syntax error tasks: 0
Successful tasks: 69
Accuracy: 100.00%
Tasks with errors: []
Tasks with syntax errors: []
